In [3]:
import numpy as np

import meshplot as mp

from copy import deepcopy

import open3d as o3d
from open3d.geometry import LineSet
from open3d.io import read_point_cloud
from open3d.visualization import draw_geometries

import pyvista as pv

from trimesh import Trimesh

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertices, myMesh.faces,c=cmap)

def visualize(pc1, pc2):
    o3d.visualization.draw_geometries([pc1, pc2])

def copy(pc1, pc2, tr1=None, tr2=None):
    tr1 = (0, 0, 0) if tr1 is None else tr1
    tr2 = (0, 0, 0) if tr2 is None else tr2
    pc1 = deepcopy(pc1).translate(tr1)
    pc2 = deepcopy(pc2).translate(tr2)
    pc1.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(pc1.points), 1)))
    pc2.colors = o3d.utility.Vector3dVector(np.tile([0, 0, 1], (len(pc1.points), 1)))
    return pc1, pc2

def double_plot(pc1: o3d.geometry.PointCloud, pc2: o3d.geometry.PointCloud):
    pc1, pc2 = copy(pc1, pc2, (0.06, 0, 0), (0, 0.06, 0)) 
    visualize(pc1, pc2)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

In [118]:
import pandas as pd
import os
df=pd.read_csv("./data/cube_dataset_original.csv")
test_samples=df.loc[df["set"]=="test"]
nondeformed=test_samples["input"].tolist()
deformed=test_samples["target"].tolist()
predicted_path="."
preds=[ (os.path.join(predicted_path,p),int(p.split("/")[-1].split("_")[0]))for p in os.listdir(predicted_path)]
preds.sort(key=lambda x: x[1])

In [122]:
for i in range(len(deformed)):
    data_path=deformed[i].split("/")
    data_dir=data_path[-2]
    file_name=data_path[-1]
    
    if not os.path.exists(os.path.join("predictions",data_dir)):
        os.makedirs(os.path.join("predictions",data_dir))
        os.makedirs(os.path.join("predictions",data_dir,"nondeformed"))
        os.makedirs(os.path.join("predictions",data_dir,"deformed"))
        os.makedirs(os.path.join("predictions",data_dir,"predicted"))
        mesh = o3d.io.read_triangle_mesh(nondeformed[i])
        mesh = o3d.geometry.TriangleMesh.compute_triangle_normals(mesh)
        o3d.io.write_triangle_mesh(os.path.join("predictions",data_dir,"nondeformed","0.stl"), mesh)
    mesh = o3d.io.read_triangle_mesh(deformed[i])
    mesh = o3d.geometry.TriangleMesh.compute_triangle_normals(mesh)
    
    o3d.io.write_triangle_mesh(os.path.join("predictions",data_dir,"deformed",file_name), mesh)

    
    mesh.remove_duplicated_vertices()
    pc = o3d.io.read_point_cloud(preds[i][0])
    mesh.vertices = pc.points
    mesh_new = o3d.geometry.TriangleMesh.compute_triangle_normals(mesh)
    o3d.io.write_triangle_mesh(os.path.join("predictions",data_dir,"predicted",file_name), mesh_new)
    
